In [1]:
import numpy as np

In [2]:
# initialize tensors
d = 2
chi = 10

A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)
sAB = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights
sBA = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights

In [3]:
from numpy import linalg as LA
from ncon import ncon

""" Contract infinite MPS from the left for environment tensor sigBA """

sigBA = np.random.rand(chi, chi)  # initialize random starting point
tol = 1e-10  # set convergence tolerance

# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]

for k in range(1000):
    sigBA_new = ncon([sigBA, *tensors], labels)  # contract transfer operator
    sigBA_new = sigBA_new / np.trace(sigBA_new)  # normalize
    if LA.norm(sigBA - sigBA_new) < tol:  # check convergence
        print('success!')
        break
    else:
        print('iter: %d, diff: %e' % (k, LA.norm(sigBA - sigBA_new)))
        sigBA = sigBA_new

iter: 0, diff: 4.496874e+00
iter: 1, diff: 4.528362e-03
iter: 2, diff: 5.250502e-05
iter: 3, diff: 5.025410e-07
iter: 4, diff: 6.340414e-09
success!


In [4]:
from scipy.sparse.linalg import LinearOperator

In [5]:
from scipy.sparse.linalg import eigs

In [6]:
def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors], labels).reshape([chiBA**2, 1])

    # returns k=1 eigenvalues and eigenvectors
    # matvec implements A * v
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB

In [7]:
sigBA = np.random.rand(chi, chi)

In [8]:
sigBA

array([[0.95455676, 0.62623088, 0.0564084 , 0.35819321, 0.72293479,
        0.73077033, 0.05391388, 0.34051448, 0.7392256 , 0.80723234],
       [0.98675348, 0.16025666, 0.26746691, 0.31021781, 0.87063312,
        0.06854128, 0.08789196, 0.28031262, 0.96311529, 0.29739678],
       [0.06462816, 0.1201552 , 0.40433535, 0.82348408, 0.97232616,
        0.7960752 , 0.99902131, 0.0794516 , 0.98774415, 0.04208947],
       [0.58472623, 0.09878602, 0.92716626, 0.61490326, 0.98846357,
        0.05960269, 0.34197886, 0.20864266, 0.2792905 , 0.08058641],
       [0.67751003, 0.12257866, 0.472352  , 0.13440697, 0.75097794,
        0.66876911, 0.25633013, 0.23103442, 0.42568718, 0.20268206],
       [0.0661882 , 0.97162731, 0.89770292, 0.53550586, 0.10997877,
        0.7382214 , 0.14222634, 0.82364979, 0.8477951 , 0.59756476],
       [0.23795013, 0.67191928, 0.04208624, 0.02484244, 0.44954518,
        0.80408349, 0.16211135, 0.82278506, 0.89860838, 0.54655322],
       [0.03485197, 0.10104319, 0.9710572

In [9]:
sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)

In [10]:
sigBA

array([[0.10459772, 0.09925522, 0.09728806, 0.10109121, 0.10123878,
        0.1077838 , 0.103775  , 0.09969849, 0.09537532, 0.09426733],
       [0.09925522, 0.09464928, 0.09337473, 0.09729719, 0.09622294,
        0.10448108, 0.09912571, 0.09517136, 0.0926423 , 0.08987863],
       [0.09728806, 0.09337473, 0.09333054, 0.09764609, 0.09451691,
        0.10603619, 0.09819265, 0.09433021, 0.09426228, 0.08894561],
       [0.10109121, 0.09729719, 0.09764609, 0.10238451, 0.0982535 ,
        0.11169753, 0.10246891, 0.09840361, 0.09940734, 0.09277984],
       [0.10123878, 0.09622294, 0.09451691, 0.0982535 , 0.0980797 ,
        0.10494678, 0.1006314 , 0.09666976, 0.09292292, 0.09132214],
       [0.1077838 , 0.10448108, 0.10603619, 0.11169753, 0.10494678,
        0.12320805, 0.11044481, 0.10601271, 0.1099455 , 0.09985183],
       [0.103775  , 0.09912571, 0.09819265, 0.10246891, 0.1006314 ,
        0.11044481, 0.10397862, 0.09985192, 0.09800222, 0.09429292],
       [0.09969849, 0.09517136, 0.0943302

In [11]:
sigAB

array([[0.0918883 , 0.09372713, 0.09565027, 0.095036  , 0.09052364,
        0.07544679, 0.09347988, 0.09398968, 0.08666402, 0.11034449],
       [0.09372713, 0.10887957, 0.10401041, 0.1047147 , 0.10687995,
        0.08659141, 0.10088873, 0.10366581, 0.09455955, 0.11982812],
       [0.09565027, 0.10401041, 0.10285664, 0.10273663, 0.10135705,
        0.08331369, 0.10004185, 0.10165565, 0.09328235, 0.11841852],
       [0.095036  , 0.1047147 , 0.10273663, 0.10287515, 0.10214746,
        0.08367557, 0.09990623, 0.10179068, 0.09325675, 0.11838685],
       [0.09052364, 0.10687995, 0.10135705, 0.10214746, 0.10514635,
        0.08493331, 0.09819239, 0.1011435 , 0.09215916, 0.11668849],
       [0.07544679, 0.08659141, 0.08331369, 0.08367557, 0.08493331,
        0.06902348, 0.08083189, 0.08284351, 0.07567977, 0.09590051],
       [0.09347988, 0.10088873, 0.10004185, 0.09990623, 0.09819239,
        0.08083189, 0.0974668 , 0.09886938, 0.09076149, 0.11530464],
       [0.09398968, 0.10366581, 0.1016556

In [12]:
def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """

    # diagonalize left environment
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A

Define the Hamiltonian.

In [13]:
from scipy.linalg import expm

# define a Hamiltonian (here the quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = hamAB

# exponentiate Hamiltonian
tau = 0.1  # set time-step
evotype = 'imag'  # set imaginary time evolution
if evotype == "real":
    gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
elif evotype == "imag":
    gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

In [14]:
gateAB

array([[[[ 1.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  1.02006676],
         [-0.201336  ,  0.        ]]],


       [[[ 0.        , -0.201336  ],
         [ 1.02006676,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  1.        ]]]])

In [15]:
def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B

Putting it all together:

In [16]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm
from scipy.sparse.linalg import LinearOperator, eigs
from ncon import ncon
from typing import Optional


def doTEBD(hamAB: np.ndarray,
           hamBA: np.ndarray,
           A: np.ndarray,
           B: np.ndarray,
           sAB: np.ndarray,
           sBA: np.ndarray,
           chi: int,
           tau: float,
           evotype: Optional[str] = 'imag',
           numiter: Optional[int] = 1000,
           midsteps: Optional[int] = 10,
           E0: Optional[float] = 0.0):
    """
    Implementation of time evolution (real or imaginary) for MPS with 2-site unit
    cell (A-B), based on TEBD algorithm.
    Args:
    hamAB: nearest neighbor Hamiltonian coupling for A-B sites.
    hamBA: nearest neighbor Hamiltonian coupling for B-A sites.
    A: MPS tensor for A-sites of lattice.
    B: MPS tensor for B-sites of lattice.
    sAB: vector of weights for A-B links.
    sBA: vector of weights for B-A links.
    chi: maximum bond dimension of MPS.
    tau: time-step of evolution.
    evotype: set real (evotype='real') or imaginary (evotype='imag') evolution.
    numiter: number of time-step iterations to take.
    midsteps: number of time-steps between re-orthogonalization of the MPS.
    E0: specify the ground energy (if known).
    Returns:
    np.ndarray: MPS tensor for A-sites;
    np.ndarray: MPS tensor for B-sites;
    np.ndarray: vector sAB of weights for A-B links.
    np.ndarray: vector sBA of weights for B-A links.
    np.ndarray: two-site reduced density matrix rhoAB for A-B sites
    np.ndarray: two-site reduced density matrix rhoAB for B-A sites
    """
    # exponentiate Hamiltonian
    d = A.shape[1]
    if evotype == "real":
        gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
    elif evotype == "imag":
        gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

    # initialize environment matrices
    sigBA = np.eye(A.shape[0]) / A.shape[0]
    muAB = np.eye(A.shape[2]) / A.shape[2]

    for k in range(numiter + 1):
        if np.mod(k, midsteps) == 0 or (k == numiter):
#             """ Bring MPS to normal form """

#             # contract MPS from left and right
#             sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)
#             muAB, muBA = right_contract_MPS(muAB, sBA, A, sAB, B)

#             # orthogonalise A-B and B-A links
#             B, sBA, A = orthog_MPS(sigBA, muBA, B, sBA, A)
#             A, sAB, B = orthog_MPS(sigAB, muAB, A, sAB, B)

#             # normalize the MPS tensors
#             A_norm = np.sqrt(ncon([np.diag(sBA**2), A, np.conj(A), np.diag(sAB**2)],
#                             [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
#             A = A / A_norm
#             B_norm = np.sqrt(ncon([np.diag(sAB**2), B, np.conj(B), np.diag(sBA**2)],
#                             [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
#             B = B / B_norm

            """ Compute energy and display """

            # compute 2-site local reduced density matrices
            rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)

            # evaluate the energy
            energyAB = ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energyBA = ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energy = 0.5 * (energyAB + energyBA)

            chitemp = min(A.shape[0], B.shape[0])
            enDiff = energy - E0
            print('iteration: %d of %d, chi: %d, t-step: %f, energy: %f, '
                'energy error: %e' % (k, numiter, chitemp, tau, energy, enDiff))

        """ Do evolution of MPS through one time-step """
        if k < numiter:
            # apply gate to A-B link
            A, sAB, B = apply_gate_MPS(gateAB, A, sAB, B, sBA, chi)

            # apply gate to B-A link
            B, sBA, A = apply_gate_MPS(gateBA, B, sBA, A, sAB, chi)

    rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)
    
    return A, B, sAB, sBA, rhoAB, rhoBA


def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
                    labels).reshape([chiBA**2, 1])
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB


def right_contract_MPS(muAB, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the right for the environment
    density matrices muAB (A-B link) and muBA (B-A link)"""

    # initialize the starting vector
    chiAB = A.shape[2]
    if muAB.shape[0] == chiAB:
        v0 = muAB.reshape(np.prod(muAB.shape))
    else:
        v0 = (np.eye(chiAB) / chiAB).reshape(chiAB**2)

    # define network for transfer operator contract
    tensors = [np.diag(sAB), np.diag(sAB), A, A.conj(), np.diag(sBA),
             np.diag(sBA), B, B.conj()]
    labels = [[1, 2], [3, 1], [5, 2], [6, 4, 3], [7, 4, 5], [8, 6], [10, 7],
            [-1, 9, 8], [-2, 9, 10]]

    # define function for boundary contraction and pass to eigs
    def right_iter(muAB):
        return ncon([muAB.reshape([chiAB, chiAB]), *tensors],
                    labels).reshape([chiAB**2, 1])
    Dtemp, muAB = eigs(LinearOperator((chiAB**2, chiAB**2), matvec=right_iter),
                     k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix muAB
    if np.isrealobj(A):
        muAB = np.real(muAB)
    muAB = muAB.reshape(chiAB, chiAB)
    muAB = 0.5 * (muAB + np.conj(muAB.T))
    muAB = muAB / np.trace(muAB)

    # compute density matrix muBA for B-A link
    muBA = ncon([muAB, np.diag(sAB), np.diag(sAB), A, A.conj()],
              [[1, 2], [3, 1], [5, 2], [-1, 4, 3], [-2, 4, 5]])
    muBA = muBA / np.trace(muBA)

    return muAB, muBA


def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """
    # diagonalize left environment matrix
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment matrix
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A


def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B


def loc_density_MPS(A, sAB, B, sBA):
    """ Compute the local reduced density matrices from an MPS (assumend to be
    in canonical form)."""

    # recast singular weights into a matrix
    mAB = np.diag(sAB)
    mBA = np.diag(sBA)

    # contract MPS for local reduced density matrix (A-B)
    tensors = [np.diag(sBA**2), A, A.conj(), mAB, mAB, B, B.conj(),
             np.diag(sBA**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoAB = ncon(tensors, connects)

    # contract MPS for local reduced density matrix (B-A)
    tensors = [np.diag(sAB**2), B, B.conj(), mBA, mBA, A, A.conj(),
             np.diag(sAB**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoBA = ncon(tensors, connects)

    return rhoAB, rhoBA

In [17]:
import numpy as np
from ncon import ncon

""" Example 1: XX model """

# set bond dimensions and simulation options
chi = 16  # bond dimension
tau = 0.1  # timestep

numiter = 500  # number of timesteps
evotype = "imag"  # real or imaginary time evolution
E0 = -4 / np.pi  # specify exact ground energy (if known)
midsteps = int(1 / tau)  # timesteps between MPS re-orthogonalization

# define Hamiltonian (quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
sZ = np.array([[1, 0], [0, -1]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)

# initialize tensors
d = hamAB.shape[0]
sAB = np.ones(chi) / np.sqrt(chi)
sBA = np.ones(chi) / np.sqrt(chi)
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

""" Imaginary time evolution with TEBD """
# run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep
tau = 0.01
numiter = 2000
midsteps = 100
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep and increased bond dim
chi = 32
tau = 0.001
numiter = 20000
midsteps = 1000
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# compare with exact results
energyMPS = np.real(0.5 * ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                    0.5 * ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]]))
enErr = abs(energyMPS - E0)
print('Final results => Bond dim: %d, Energy: %f, Energy Error: %e' %
      (chi, energyMPS, enErr))

iteration: 0 of 500, chi: 16, t-step: 0.100000, energy: 4.119610, energy error: 5.392850e+00
iteration: 10 of 500, chi: 16, t-step: 0.100000, energy: -1.229462, energy error: 4.377777e-02
iteration: 20 of 500, chi: 16, t-step: 0.100000, energy: -1.237674, energy error: 3.556555e-02
iteration: 30 of 500, chi: 16, t-step: 0.100000, energy: -1.238373, energy error: 3.486627e-02
iteration: 40 of 500, chi: 16, t-step: 0.100000, energy: -1.238516, energy error: 3.472363e-02
iteration: 50 of 500, chi: 16, t-step: 0.100000, energy: -1.238531, energy error: 3.470857e-02
iteration: 60 of 500, chi: 16, t-step: 0.100000, energy: -1.238513, energy error: 3.472692e-02
iteration: 70 of 500, chi: 16, t-step: 0.100000, energy: -1.238487, energy error: 3.475223e-02
iteration: 80 of 500, chi: 16, t-step: 0.100000, energy: -1.238463, energy error: 3.477636e-02
iteration: 90 of 500, chi: 16, t-step: 0.100000, energy: -1.238443, energy error: 3.479699e-02
iteration: 100 of 500, chi: 16, t-step: 0.100000, en

iteration: 13000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272793, energy error: 4.466304e-04
iteration: 14000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272794, energy error: 4.459306e-04
iteration: 15000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272794, energy error: 4.453992e-04
iteration: 16000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.449944e-04
iteration: 17000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.446855e-04
iteration: 18000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.444494e-04
iteration: 19000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.442685e-04
iteration: 20000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.441300e-04
Final results => Bond dim: 32, Energy: -1.272795, Energy Error: 4.441300e-04
